### Downlaod from quran.gov.bd

In [ ]:
using System.IO;
using System.Text.Json;
using System.Net;

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

record DownloadableFile(string url, string filename);

var downlaodableFiles = new List<DownloadableFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;
    //var start = (int)sura[0];
    var ayas = int.Parse(sura[1].ToString());

    for(int j = 1; j <= ayas; j++){
        downlaodableFiles.Add( new($"{serial}/{serial}-{j}.mp3", $"{serial.ToString().PadLeft(3, '0')}{j.ToString().PadLeft(3, '0')}.mp3"));
    }
}

var urlBase = "http://quran.gov.bd/quran/Sound/bangla";
var downloadPath = @"D:\Faruque\Projects\QuranAudio\mahmudur_rahman_quran.gov.bd";

foreach (var downlaodableFile in downlaodableFiles){
    string url = $"{urlBase}/{downlaodableFile.url}";
    string filePath = $"{downloadPath}\\{downlaodableFile.filename}";

    if (File.Exists(filePath))
        continue;

    using (var client = new WebClient())
    {
        client.DownloadFile(url, filePath);
    }
    Console.WriteLine($"Downloaded file: {downlaodableFile}");
}

Console.WriteLine($"Download completed");
